# 네트워크 분석
많은 데이터 문제는 노드(node)와 그 사이를 연결하는 엣지(edge)로 구성된 네트워크(network)의 관점에서 해석할 수 있다.

e.g. 페이스북은 사용자가 노드, 친구 관계가 엣지. 웹에서 각 웹페이지가 노드 페이지 사이를 연결하는 하이퍼링크가 엣지.

페이스북에서는 친구 관계는 상호적이다. 즉, 내가 누군가와 친구이면 그 사람은 반드시 나와 친구이다. 이런 경우를 엣지에 **방향이 없다(undirected)** 고 한다. 반면 하이퍼링크는 그렇지 않다. 링크가 존재 할 수도 안할 수도 있다. 이런 네트워크는 방향이 있기 때문에 **방향성 네트워크(directed network)** 라 한다.

## 22.1 매개 중심성

In [1]:
from typing import NamedTuple

class User(NamedTuple):
    id: int
    name: str

users = [User(0, "Hero"), User(1, "Dunn"), User(2, "Sue"), User(3, "Chi"),
         User(4, "Thor"), User(5, "Clive"), User(6, "Hicks"),
         User(7, "Devin"), User(8, "Kate"), User(9, "Klein")]

friend_pairs = [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (3, 4),
                (4, 5), (5, 6), (5, 7), (6, 8), (7, 8), (8, 9)]


친구 관계를 딕셔너리로 다루면 더 간편하다.

In [2]:
from typing import Dict, List

# 친구 관계를 유지하기 위한 type alias
Friendships = Dict[int, List[int]]

friendships: Friendships = {user.id: [] for user in users}

for i, j in friend_pairs:
    friendships[i].append(j)
    friendships[j].append(i)

assert friendships[4] == [3, 5]
assert friendships[8] == [6, 7, 9]

앞에서 연결 중심성(degree centrality)을 살펴볼때는, 선정된 핵심 인물이 우리의 직관과 일치하지 않아서 아쉬웠다.

대안으로 사용할 수 있는 지수는 **매개 중심성(betweenness cnetrality)** 인데, 이는 임의의 두 사람 사이의 최단 경로상에 얼마나 자주 등장하는지를 보는 지수이다. 구체적으로는 노드 i의 매개 중심성은 다른 모든 노드 j,k쌍 사이의 최단 경로 중에, i를 겨치는 경로의 비율로 계산한다.

e.g. Thor의 매개 중심성을 구하려면 Thor가 아닌 다른 모든 사람 간의 최단 경로를 구해야 한다. 그리고 그중 어떤 경로들이 Thor를 거쳐 가는지 확인한다. e.g Chi(id 3)와 Clive(id 5) 사이의 최단 경로는 Thor를 거치지만 Hero(id 0)과 Chi(id 3) 사이의 최단 경로 두 개는 Thor를 거치지 않는다.

결국, 임의의 두 사람이 주어졌을 때 그들 간의 최단 경로를 모두 구할 수 있어야 한다. 최단 경로를 효율적으로 구해주는 복잡한 방법이 많이 있지만, 여기서는 이해하기 쉬운 알고리즘을 사용할 것이다.

'너비 우선 탐색(breadth-first search)'라고 알려진 이 알고리즘은 아래와 같다.

1. 목표는 from_user를 받아서 다른 모든 사용자까지의 최단 경로를 계산해 주는 함수를 만드는 것이다.

2. 각 경로는 사용자 ID의 리스트로 표현할 것이다. 모든 경로는 from_user로부터 시작하기 때문에 이는 따로 리스트에 추가할 필요가 없다. 그러므로 리스트의 길이는 경로의 길이를 나타낸다. 

3. 사용자 ID를 키(key)로 사용하고, 해당 사용자에 도달할 때까지의 경로를 나타내는 리스트들을 값으로 사용하는 딕셔너리인 shortest_paths_to를 유지할 것이다. 만약 유일한 최단 경로가 존재한다면 이 리스트는 해당 최단 경로만 포함하고 있을 것이고, 만약 여러 최단 경로가 존재한다면 이 리스트는 모든 최단 경로를 포함하고 있을 것이다.

4. frontier라는 큐(queue)를 사용하여 살펴보고 싶은 사용자의 순서를 유지할 것이다. 이전 사용자에서 다음 사용자까지 어떻게 도달했는지 나타내기 위해 (prev_user, user) 형태의 데이터를 큐에 넣어서 초기화한다.

5. 네트워크를 살펴보면서 최단 경로를 모르는 새로운 이웃이 보이면 현재 사용자를 prev_user로 설정하고, 새로운 이웃을 큐의 맨 끝에 추가해 주자.

6. 특정 사용자까지의 최단 경로를 계산한 적이 없지만 해당 사용자가 큐에서 제거된다면, 해당 사용자에 도달할 수 있는 최단 경로를 찾았다는 것을 의미한다. prev_user에 도달하는 최단 경로에서 해당 사용자 방향으로 한 번 더 나아간 경로가 해당 사용자까지의 최단 경로이다.

7. 큐에서 사용자를 제거할 때 해당 사용자까지의 최단 경로를 이미 계산했다면, 새로운 경로를 찾았다는 것을 의미한다. 만약 찾은 경로가 최단 경로라면 추가하고 아니면 추가하지 말자.

8. 만약 큐에 더 이상 사용자가 남아 있지 않다면 네트워크 전체를 살펴봤다는 것을 의미한다(적어도 시작점에서 도달할 수 있는 모든 경로를 살펴봤다는 것을 의미한다). 이제 끝이다.

위의 과정들을 큰 함수로 만들어 보자

In [3]:
from collections import deque
Path = List[int]

def shortest_paths_from(from_user_id: int,
                       friendships: Friendships) -> Dict[int, List[Path]]:
    # 특정 사용자로부터 다른 사용자까지의 *모든* 최단 경로를 포함하는 dict
    shortest_paths_to: Dict[int, List[Path]] = {from_user_id: [[]]}
        
    # 확인해야 하는 (이전 사용자, 다음 사용자) 큐
    # 모든 (from_user, from_user의 친구) 쌍으로 시작
    frontier = deque((from_user_id, friend_id)
                    for friend_id in friendships[from_user_id])
    
    # 큐가 빌 때까지 반복
    while frontier:
        # 큐의 첫 번째 사용자를 제거
        prev_user_id, user_id = frontier.popleft()
        
        
        # 큐에 사용자를 추가하는 방법을 고려해 보면
        # prev_user까지의 최단 경로의 몇 개를 이미 알고 있다.
        paths_to_prev_user = shortest_paths_to[prev_user_id]
        new_paths_to_user = [path + [user_id] for path in paths_to_prev_user]
        
        # 만약 최단 경로를 이미 알고 있다면
        old_paths_to_user = shortest_paths_to.get(user_id, [])
        
        # 지금까지 최단 경로는 무엇일까?
        if old_paths_to_user:
            min_path_length = len(old_paths_to_user[0])
        else:
            min_path_length = float('inf')
            
        # 길지 않은 새로운 경로만 저장
        new_paths_to_user = [path for path in new_paths_to_user
                            if len(path) <= min_path_length
                            and path not in old_paths_to_user]
        shortest_paths_to[user_id] = old_paths_to_user + new_paths_to_user
        
        # 아직 한 번도 보지 못한 이웃을 frontier에 추가
        frontier.extend((user_id, friend_id)
                       for friend_id in friendships[user_id]
                       if friend_id not in shortest_paths_to)
        
    return shortest_paths_to

이제 모든 최단거리를 계산해 보자

In [4]:
# 각 from_user에서 각 to_user까지 최단 경로 리스트를 반환 
shortest_paths = {user.id: shortest_paths_from(user.id, friendships)
                 for user in users}

이제 매개 중심성을 구할 준비가 되었다. 모든 노드 쌍 i,j에 대해 i부터 j까지 n개의 최단 경로를 알고 있다. 이제 각각의 최단 경로에 포함되는 각 노드의 매개 중심성에 1/n을 더해 주자.

In [5]:
betweenness_centrality = {user.id: 0.0 for user in users}

for source in users:
    for target_id, paths in shortest_paths[source.id].items():
        if source.id < target_id:      # 잘못해서 두 번 세지 않도록 주의하자.
            num_paths = len(paths)     # 최단 경로가 몇 개 존재하는가?
            contrib = 1 / num_paths    # 중심성에 기여하는 값
            for path in paths:
                for between_id in path:
                    if between_id not in [source.id, target_id]:
                        betweenness_centrality[between_id] += contrib

그 외에 살펴볼 수 있는 중심성 지표 중 하나는 **근접 중심성(closeness centrality)** 이다. 먼저 각 사용자의 **원접성(farness)** 을 계산한다. 원접성이란 어떤 사용자와 다른 모든 사용자의 최단 경로를 합한 값이다. 이미 모든 노드의 최단 경로를 계산했으니, 이를 더하는 것은 어렵지 않다. (만약 여러 개의 최단 경로를 찾았다면 어차피 모든 최단 경로의 거리는 동일하니 첫 번째 최단 경로의 거리만 더해 주자).

In [8]:
def farness(user_id: int) -> float:
    """모든 사용자와의 최단 거리 합"""
    return sum(len(paths[0])
              for paths in shortest_paths[user_id].values())

In [9]:
closeness_centrality = {user.id: 1 / farness(user.id) for user in users}

In [10]:
closeness_centrality

{0: 0.029411764705882353,
 1: 0.037037037037037035,
 2: 0.037037037037037035,
 3: 0.045454545454545456,
 4: 0.05,
 5: 0.05,
 6: 0.041666666666666664,
 7: 0.041666666666666664,
 8: 0.03571428571428571,
 9: 0.027777777777777776}

근접 중심성의 편차는 훨씬 작다. 네트워크 중심에 있는 노드조차 외곽에 위치한 노드들로부터 멀리 떨어져 있기 때문이다. 

여기서 봤듯이 최단 경로를 계산하는 것은 꽤나 복잡하다. 그렇기 때문에 큰 네트워크에서는 연결 중심성 및 근접 중심성을 자주 사용하지 않는다. 덜 직관적이지만 보통 더 쉽게 게산할 수 있는 **고유벡터 중심성(eigenvector centrality)** 을 더 자주 사용한다.

## 22.2 고유벡터 중심성
고유벡터 중심성에 대해 알아보기 전 고유벡터가 무엇인지 알아야 한다. 고유벡터가 무엇인지 알기 위해서는 먼저 행렬 곱셈에 대해 알아봐야 한다.

### 22.2.1 행렬 곱셈
A가 n x m행렬, B가 m x k 행렬 (A의 두 번째 차원과 B의 첫 번째 차원이 같다)이면 두 행렬의 곱 AB는 n x k 행렬이 되고 이때 (i, j)번째 항목의 값은 다음과 같다.

$A_i1 B_1j + A_i2 B_2j + ... + A_im B_mj$

이는 A의 i번 째 행과 B의 j번 째 열의 내적과 동일하다.

이는 4장 선형대수에서 나온 make_matrix 함수를 사용하여 구현할 수 있다.

In [14]:
from typing import Callable

Matrix = List[List[float]]

def make_matrix(num_rows: int, num_cols: int, entry_fn: Callable[[int, int], float]) -> Matrix:
    """(i, j)번째 원소가 entry_fn(i, j)인 num_rows x num_cols 리스트를 반환"""
    return [[entry_fn(i, j) for j in range(num_cols)] for i in range(num_rows)]

from typing import Tuple

def shape(A: Matrix) -> Tuple[int, int]:
    """(열의 개수, 행의 개수)를 반환"""
    num_rows = len(A)
    num_cols = len(A[0]) if A else 0  # 첫 번째 행의 원소의 개수
    return num_rows, num_cols

In [16]:
def matrix_times_matrix(m1: Matrix, m2: Matrix) -> Matrix:
    nr1, nc1 = shape(m1)
    nr2, nc2 = shape(m2)
    assert nc1 == nr2, "must have (# of columns in m1) == (# of rows in m2)"

    def entry_fn(i: int, j: int) -> float:
        """m1의 i 번째 행과 m2의 j 번째 열의 내적"""
        return sum(m1[i][k] * m2[k][j] for k in range(nc1))

    return make_matrix(nr1, nc2, entry_fn)

m차원 벡터를 (m, 1) 행렬로 생각하면, 이를 (n, m) 행렬과 곱하여 n차원 벡터로 간주할 수 있는 (n, 1) 행렬을 얻을 수 있다.

즉, (n, m) 행렬을, m차원 벡터를 n차원 벡터로 변환하는 선형 사상(linear mapping)으로 생각해 볼 수 있음을 의미한다.

In [17]:
from typing import List

Vector = List[float]

def dot(v: Vector, w: Vector) -> float:
    """v_1 * w_1 + ... + v_n * w_n"""
    assert len(v) == len(w),  "vectors must be same length"
    
    return sum(v_i * w_i for v_i, w_i in zip(v,w))

In [18]:
def matrix_times_vector(m: Matrix, v: Vector) -> Vector:
    nr, nc = shape(m)
    n = len(v)
    assert nc == n, "must have (# of cols in m) == (# of elements in v)"
    
    return [dot(row, v) for row in m]  # outputs has length nr

A가 **사각 행렬(square matrix)** 이라면 이 연산은 n차원 벡터를 다른 n차원 벡터로 변환시킨다. 때로는 A와 v의 값에 따라 A와 v를 곱하면 v의 상수 배에 해당하는 벡터를 얻을 때가 있다. 즉, 결괏값으로 나온 벡터는 기존 벡터 v와 같은 방향을 가리키는 벡터가 될 때가 있다. v가 0으로만 이뤄진 벡터가 아닌데도 이런 일이 생기면 v를 A의 고유벡터(eigenvector)라고 부르고, 이때 곱해진 상수를 **고윳값(eigenvalue)** 이라고 부르게 된다.

행렬 A의 고유벡터를 찾는 한 가지 방법은, 임의의 벡터 v를 골라 matrix_times_vector를 수행하고, 결괏값의 크기가 1이 되게 재조정하는 과정을 반복 수행하는 것이다. 

In [19]:
from typing import Tuple
import random
import math

def dot(v: Vector, w: Vector) -> float:
    """v_1 * w_1 + ... + v_n * w_n"""
    assert len(v) == len(w),  "vectors must be same length"
    
    return sum(v_i * w_i for v_i, w_i in zip(v,w))

def sum_of_squares(v: Vector) -> float:
    """v_1 * v_1 + ... v_n * v_n"""
    return dot(v,v)

def magnitude(v: Vector) -> float:
    """벡터 v의 크기를 반환"""
    return math.sqrt(sum_of_squares(v))

def squared_distance(v: Vector, w: Vector) -> float:
    """(v_1 - w_1)**2 + ... + (v_n - w_n)**2"""
    return sum_of_squares(subtract(v, w))

def distance(v: Vector, w: Vector) -> float:
    """벡터 v와 w 간의 거리를 계산"""
    return math.sqrt(squared_distance(v,w))

In [20]:
def find_eigenvector(m: Matrix,
                    tolerance: float = 0.00001) -> Tuple[Vector, float]:
    guess = [random.random() for _ in m]
    
    while True:
        result = matrix_times_vector(m, guess)  # guess를 변환
        norm = magnitude(result)  # 크기를 계산
        next_guess = [x / norm for x in result]  # 재조정
        
        if distance(guess, next_guess) < tolerance:
            # 수렴했으니 (고유벡터, 고윳값)으로 반환
            return next_guess, norm
        
        guess = next_guess

반환된 guess에 matrix_times_vector를 적용하고 크기가 1인 벡터로 재조정을 하면 자기 자신과 매우 가까운 벡터를 얻을 수 있다. 즉, guess는 고유벡터이다. 

모든 실수 행렬에 고유벡터와 고윳값이 존재하는 것은 아니다. 다음 행렬을 살펴보자.

rotate = [[0, 1],
          [-1, 0]]
          
이 행렬은 벡터를 시계 방향으로 90도 회전하는 연산을 수행하기 때문에 해당 행렬에 곱했을 때 자기 자신이 되는 벡터는 영벡터 밖에 없다. 이 행렬로 앞서 구현한 find_eigenvector(rotate)를 수행하면 영원히 끝나지 않을 것이다. 한편, 고유벡터가 있는 행렬도 때로는 무한루프에 빠질 수 있다. 다음 행렬을 고려해 보자.

flip = [[0, 1], [1, 0]]

이 행렬은 모든 벡터 [x, y]를 [y, x]로 변환한다. 따라서 [1, 1]은 해당 행렬의 고유벡터이며 이때 고윳값은 1이 된다. 하지만 x, y값이 서로 다른 임의의 벡터에서 출발해서 find_eigenvector를 수행하면 두 값을 바꾸는 연산만 무한히 수행할 것이다. 

### 22.2.2 중심성
네트워크를 인접행렬(adjacency matrix)의 형태로 나태내 보자. 이 행렬은 사용자 i와 사용자 j가 친구인 경우 (i, j)번째 항목에 1이 있고, 친구가 아닌 경우 0이 있는 행렬이다.

In [21]:
def entry_fn(i: int, j: int):
    return 1 if (i, j) in friend_pairs or (j, i) in friend_pairs else 0
n = len(users)
adjacency_matrix = make_matrix(n, n, entry_fn)

각 사용자의 고유벡터 중심성이란 find_eigenvector로 찾은 고유벡터에서 각 사용자에 대응되는 원소가 된다.

eigenvector_centralities, _ = find_eigenvector(adjacency_matrix)

연결의 수가 많고, 중심성이 높은 사용자들한테 연결된 사용자들은 고유벡터 중 심성이 높다.

앞의 결과에 따르며면 사용자 1, 사용자 2의 중심성이 가장 높은데, 이는 중심성이 높은 사람들과 세 번이나 연결되었기 때문이다. 이들로부터 멀어질수록 사용자들의 중심성은 점차 줄어든다.

이렇게 작은 네트워크에서 고유벡터 중심성은 다소 불규칙적으로 작동한다. 

연결을 하나만 더하거나 빼더라도 중심성에는 큰 변화가 생긴다. 하지만 네트워크의 크기가 커지면 이런 문제는 사라진다.

고유벡터는 다음의 값을 계산했을 때 결괏값이 eigenvector_centralities의 상수 배가 된다.

matrix_times_vector(adjacency_matrix, eigenvector_centralities)

행렬곱이 너떻게 동작하는지 살펴보면 matrix_times_vector는 i번째 항목으로 다음의 값을 가지는 벡터를 계산해 준다는 것을 알 수 있다.

dot(adjacency_matrix[i], eigenvector_centralities)

이 값은 사용자 i에게 연결되어 있는 사용자들의 고유벡터 중심서의 합과 같다. 바꿔 말하면, 고유벡터 중심성은 사용자당 하나씩 부여되는 숫자인데, 그 숫자는 이웃의 중심성의 합의 상수 배이다. 이 경우에 중심성이란, 중심성이 높은 사람들과 연결괴어 있는 정도를 의미한다. 즉, 중심성이 높은 사람들과 많이 연결되어 있을수록 자신의 중심성도 높다. 순환 논리이기는 하지만, 고유벡터가 그 순환성에서 벗어나게 해준다. 

find_eigenvector의 역할에 대해 생각해 보면 이 현상을 다른 관점에서 해석할 수 있다. find_eigenvector는 각 노드에 임의의 중심성을 할당하는 것으로 시작해서, 다음 두 단계가 수렴할 때까지 반복한다.

1. 각 노드의 이웃에 있는 중심성 정수의 합으로 해당 노드에 새로운 중심성 점수를 부여한다.

2. 중심성 벡터의 크기가 1이 되도록 벡터를 재조정한다.

## 22.3 방향성 그래프와 페이지랭크
새로운 보증(endorsement) 모델을 개발, 새로운 모델에서 관계는 상호적인 것이 아니라, 한 사람(source)이 다른 한 사람(target)의 실력에 보증을 서주는 (source, target) 쌍으로 비대칭적인 관계를 표현하게 된다.

In [23]:
endorsements = [(0, 1), (1, 0), (0, 2), (2, 0), (1, 2),
                (2, 1), (1, 3), (2, 3), (3, 4), (5, 4),
                (5, 6), (7, 5), (6, 8), (8, 7), (8, 9)]

In [25]:
from collections import Counter

endorsement_counts = Counter(target for source, target in endorsements)

사실 '보증의 수'와 같은 숫자는 조작하기가 매우 쉽다. 가장 간단한 방법 중 하나는, 가짜 계정을 여러 개 만들어서 그것들로 내 계정에 대한 보증을 서는 것이다. 

좀 더 나은 지수는, 누가 보증을 서는지를 고려하는 것이다. 보증을 많이 받은 사용자가 보증을 설 때는, 보증을 적게 받은 사용자가 보증을 설 때보다 더 중요한 것으로 받아들여지는 것이 타당하다. 그리고 이것이 구글 웹사이트들 간의 순위를 매길 때 사용하는 **페이지 랭크(PageRank)** 알고리즘의 기본 철학이기도 하다. 페이지랭크는 어떤 웹사이트들이 해당 웹사이트를 링크하고 있는지를 기반으로 웹사이트의 순위를 매긴다.

알고리즘은 아래와 같다.

1. 네트워크 전체에는 1.0 (또는 100%)의 페이지랭크가 있다.

2. 초기에 이 페이지랭크를 모든 노드에 고르게 배당한다.

3. 각 스텝을 거칠 때마다 각 노드에 배당된 페이지랭크의 대부분은 외부로 향하는 링크에 균등하게 배당한다.

4. 각 스텝을 거칠 때마다 각 노드에 남아 있는 페이지랭크를 모든 노드에 고르게 배당한다.

In [26]:
import tqdm

def page_rank(users: List[User],
             endorsements: List[Tuple[int, int]],
             damping: int = 100) -> Dict[int, float]:
    # 몇 명을 보증하는지 계산
    outgoing_counts = Counter(target for source, target in endorsements)
    
    # 페이지랭크를 균등하게 배분
    num_users = len(users)
    pr = {user.id: 1 / num_users for user in users}
    
    # 각 사용자가 매번 얻는 페이지랭크의 일부
    base_pr = (1 - damping) / num_users
    
    for iter in tqdm.trange(num_iters):
        next_pr = {user.id: base_pr for user in users}  # start with base_pr
        
        for source, target in endorsements:
            # 감쇄한 source pr을 target에 더한다.
            next_pr[target] += damping * pr[source] / outgoing_counts[source]
        pr = next_pr
        
    return pr

페이지랭크를 계산하면 아래와 같다.

In [28]:
pr = page_rank(users, endorsements)

# Thor(사용자 4)가 가장 큰 페이지랭크를 지닌다.
assert pr[4] > max(page_rank
                  for user_id, page_rank in pr.items()
                  if user_id != 4)

NameError: name 'num_iters' is not defined

비록 Thor가 사용자 0, 1, 2에 비해 바은 보증의 수는 적지만 (2개), 보증을 서 준 사용자들이 보증을 많이 받았기 때문에 그 수가 Thor에게 누적이 된다. 게다가 이 사용자들은 Thor 외의 다른 사용자에게 보증을 서 주지 않은 것도 Thor의 중심성을 더 높게 하는 요인이 된다.